In [1]:
from pymongo import MongoClient
import pandas as pd
from bson import ObjectId
from location import findCityFromCoords, findCoordsFromCity

In [2]:
client = MongoClient()
db = client['companies']
companies = db['companies']
offices = db['offices']

In [4]:
#Perform a reverse geocoding query to find the city in the offices that have coordinates
startups = companies.find({'isTech':True})
available_offices = companies.find({'isAvailable':True})

for company in startups:
    for office in offices.find({'company':ObjectId(company['_id']),
                                'city':{'$in':['',None]},
                                'location.coordinates':{'$ne':[0,0]}}):
        city = findCityFromCoords(office)
        if city == 'Unknown':
            offices.delete_one(office)
        offices.update_one(office,{'$set':{'city':city}})

for company in available_offices:
    for office in offices.find({'company':ObjectId(company['_id']),
                                'city':{'$in':['',None]},
                                'location.coordinates':{'$ne':[0,0]}}):
        city = findCityFromCoords(office)
        if city == 'Unknown':
            offices.delete_one(office)
        offices.update_one(office,{'$set':{'city':city}})

In [5]:
companies.find()[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'category_code': 'enterprise',
 'founded_year': 1996.0,
 'deadpooled_year': 2.0,
 'description': 'Server Management Software',
 'total_money_raised': 0.0,
 'isAvailable': True,
 'isTech': False}

In [53]:
#Perform a forward geocoding query to find the approximated coordinates from the office adress
startups = companies.find({'isTech':True})
available_offices = companies.find({'isAvailable':True})

for company in startups:
    for office in offices.find({'company':ObjectId(company['_id']),
                                'city':{'$nin':['',None]},
                                'location.coordinates':[0,0]}):
        loc = findCoordsFromCity(office)
        if loc == 'Unknown':
            offices.delete_one(office)
        offices.update_one(office,{'$set':{'location':loc}})

for company in available_offices:
    for office in offices.find({'company':ObjectId(company['_id']),
                                'city':{'$nin':['',None]},
                                'location.coordinates':[0,0]}):
        loc = findCoordsFromCity(office)
        if loc == 'Unknown':
            offices.delete_one(office)
        offices.update_one(office,{'$set':{'location':loc}})

In [99]:
#A database with tech companies download from angellist 
#Future development

df_startups = pd.read_csv('startups.csv')
df_startups['Total Raised'] = df_startups['Total Raised'].apply(lambda s:round(s/1000000,1))
df_startups.Location.value_counts().head(5)

San Francisco    16
Los Angeles       4
New York City     4
Boston            3
Santa Monica      2
Name: Location, dtype: int64